In [ ]:
import os, sys
import json
import pickle as pk
import numpy as np
import matplotlib
import bloscpack as bp
import pandas as pd
from matplotlib import pyplot as plt
import neuroglancer

In [ ]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(os.path.join(os.getcwd(), PATH))
from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer

In [ ]:
color_map_file = os.path.join(PATH, 'contours', 'json_cache', 'struct_to_color_2.json')
with open( color_map_file, 'r') as json_file:
    structure_to_color = json.load( json_file )

stack_param_file = os.path.join(PATH, 'contours', 'json_cache', 'stack_parameters_ng.json')
with open( stack_param_file, 'r') as json_file:
    stack_parameters_ng = json.load( json_file )
    
color_segments=[]
for i in range(1,29):
    color_segments.append(i)

In [ ]:
neuroglancer.set_server_bind_address('0.0.0.0')
viewer = neuroglancer.Viewer()
print(viewer)

In [ ]:
stack = 'MD589'
target_str = '7N'

In [ ]:
filepath = os.path.join('/home/eddyod/MouseBrainSlicer_data/MD589', 'Annotation.npy')
annotation = np.load(filepath, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
hand_annotations = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})

In [ ]:
str_contours_annotation, first_sec, last_sec = get_contours_from_annotations(stack, target_str, hand_annotations, densify=0)

In [ ]:
try:
    color=structure_to_color[target_str]
except:
    color=3

ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                color_radius=2, xy_ng_resolution_um=5, threshold=1, color=color, \
                                                solid_volume=False, no_offset_big_volume=True, save_results=False, \
                                                return_with_offsets=False, add_to_ng=True, human_annotation=True )

In [ ]:
bp_path = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/MD589/MD589_wholebrainWithMargin_10.0um_intensityVolume/MD589_wholebrainWithMargin_10.0um_intensityVolume.bp'
pk_path = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_meshes/atlasV7/instance_sources'
file = 'atlasV7_7N_sources.pkl'
filepath = os.path.join(pk_path, file)

In [ ]:
MD589_bpdata = bp.unpack_ndarray_from_file(bp_path)

In [ ]:
filepath = '/home/eddyod/MouseBrainSlicer_data/v_atlas/meta_information.pkl'
with open(filepath,'rb') as pickle_file:
    presults=pk.load(pickle_file)

In [ ]:
filename = os.path.join(filepath, '/home/eddyod/MouseBrainSlicer_data/v_atlas/compressed_atlas.npy')
npdata = np.load(filename)

In [ ]:
start, finish = (300,305)
plt.style.use('classic')
for i in range(start, finish):    
    plt.figure()
    plt.imshow(npdata[:,:,i], cmap='gray')
    plt.show()